# Sprint 深層学習スクラッチ 畳み込みニューラルネットワーク2

## 【問題1】2次元畳み込み層の作成
1次元畳み込み層のクラスConv1dを発展させ、2次元畳み込み層のクラスConv2dを作成してください。


フォワードプロパゲーションの数式は以下のようになります。


$
a_{i,j,m} = \sum_{k=0}^{K-1}\sum_{s=0}^{F_{h}-1}\sum_{t=0}^{F_{w}-1}x_{(i+s),(j+t),k}w_{s,t,k,m}+b_{m}
$

In [2]:
class Conv2d:
    def __init__(self , W , B ,  stride=1 , pad=0 ):
        self.W = W
        self.B = B
        self.stride = stride
        self.pad = pad
        
        self.col = None
        self.col_W = None

        self.dW = None
        self.dB = None
        self.hw = 0
        self.hb = 0

    def _out_h(self , H , FH , pad_h=0 , stride_h=1):
        self.out_h = int(H + 2*pad_h - FH // stride_h+1)
        return self.out_h        

    def _out_w(self , W , FW , pad_w=0 , stride_w=1):
        self.out_w = int(W + 2*pad_w - FW // stride_w+1)
        return self.out_w
    
    def _im2col(self , X , FH , FW):
        N , C , H , W = X.shape
        out_h = self._out_h(H , W)
        out_w = self._out_w(W , FW)

        img = np.pad(X , [(0,0),(0,0),(self.pad,self.pad),(self.pad,self.pad)] , "constant")
        col = np.zeros((N , C , FH , FW , out_h , out_w))

        for y in range(FH):
            y_max = y + self.stride*out_h
            for x in range(FW):
                x_max = x + self.stride*out_w
                col[: , : , y , x , : , :] = img[: , : , y:y_max:self.stride, x:x_max:self.stride]

        col = col.transpose(0 , 4 , 5 , 1 , 2 , 3).reshape(N*out_h*out_w, -1)
        return col

    def _col2im(self , dcol , X , FH , FW):
        N , C , H , W = self.X.shape
        out_h = self._out_h(H , FH)
        out_w = self._out_w(W , FW)
        col = dcol.reshape(N , out_h , out_W , C , FH , FW).transpose(0,3,4,5,1,2)

        img = np.zeros(N , C , H , 2*self.pad + self.stride-1 , W + 2*self.pad + self.stride-1)
        for y in range(FH):
            y_max = y + self.stride*out_h
            for x in range(FW):
                x_max = x + self.stride*out_w
                img[: , : , y:y_max:self.stride , x:x_max:self.stride] += col[: , : , y , x , : , :]

        return img[: , : , pad:H+pad , pad:W+pad]

    def forward(self , X):
        FN , C , FH , FW = self.W.shape
        N , C , H , W = X.shape
        out_h = self._out_h(H , FH)
        out_w = self._out_w(W , FW)
        
        col = self._im2col(X , FH , FW)
        col_W = self.W.reshape(FN , -1).T
        
        A = np.dot(col , col_W) + self.B
        A = A.reshape(N , out_h , out_w , -1).transpose(0 , 3 , 1 , 2)
        
        self.X = X
        self.col = col
        self.col_W = col_W
        
        return A
    
    def backward(self , dA):
        FN , C , FH , FW = self.W.shape
        dA = dA.transpose(0 , 2 , 3 , 1).reshape(-1 , FN)
        
        self.dB = np.sum(dA , axis = 0)
        self.dW = np.dot(self.col.T , dA)
        self.dW = self.dW.transpose(1 , 0).reshape(FN , C , FH ,FW)
        
        dcol = np.dot(dA , self.col_W.T)
        dX = _col2im(dcol , X  , FH , FW)
        
        return dX

## 【問題2】2次元畳み込み後の出力サイズ
畳み込みを行うと特徴マップのサイズが変化します。どのように変化するかは以下の数式から求められます。この計算を行う関数を作成してください。

$
N_{h,out} =  \frac{N_{h,in}+2P_{h}-F_{h}}{S_{h}} + 1\\
N_{w,out} =  \frac{N_{w,in}+2P_{w}-F_{w}}{S_{w}} + 1
$

In [4]:
def out_h(H , FH , stride_h=1 , pad_h=0):
    int(H + 2*pad_h - FH // stride_h+1)
    return out_h

def out_w(W , FW , stride_w=1 , pad_w=0):
    int(W + 2*pad_w - FW // stride_w+1)
    return out_w

## 【問題3】最大プーリング層の作成
最大プーリング層のクラスMaxPool2Dを作成してください。プーリング層は数式で表さない方が分かりやすい部分もありますが、数式で表すとフォワードプロパゲーションは以下のようになります。

$
a_{i,j,k} = \max_{(p,q)\in P_{i,j}}x_{p,q,k}
$

In [5]:
class MaxPool2D:
    def __init__(self , pool_h , pool_w , stride=2 , pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.X = None
        self.arg_max = None

    def _im2col(self , X , FH , FW):
        N , C , H , W = X.shape
        out_h = (H + 2*self.pad - FH)//self.stride + 1
        out_w = (W + 2*self.pad - FW)//self.stride + 1

        img = np.pad(X , [(0,0),(0,0),(self.pad,self.pad),(self.pad,self.pad)] , "constant")
        col = np.zeros((N , C , FH , FW , out_h , out_w))

        for y in range(FH):
            y_max = y + self.stride*out_h
            for x in range(FW):
                x_max = x + self.stride*out_w
                col[: , : , y , x , : , :] = img[: , : , y:y_max:self.stride, x:x_max:self.stride]

        col = col.transpose(0 , 4 , 5 , 1 , 2 , 3).reshape(N*out_h*out_w, -1)
        return col
    
    def _col2im(self , dcol , X , FH , FW):
        N , C , H , W = X.shape
        out_h = (H + 2*self.pad - FH)//self.stride + 1
        out_w = (W + 2*self.pad - FW)//self.stride + 1
        col = dcol.reshape(N , out_h , out_w , C , FH , FW).transpose(0,3,4,5,1,2)

        img = np.zeros((N , C , H + 2*self.pad + self.stride-1 , W + 2*self.pad + self.stride-1))
        for y in range(FH):
            y_max = y + self.stride*out_h
            for x in range(FW):
                x_max = x + self.stride*out_w
                img[: , : , y:y_max:self.stride , x:x_max:self.stride] += col[: , : , y , x , : , :]

        return img[: , : , self.pad:H+self.pad , self.pad:W+self.pad]
      
    def forward(self , X):
        N , C , H , W = X.shape
        out_h = int((H - self.pool_h) // self.stride + 1)
        out_w = int((W - self.pool_w) // self.stride + 1)
        
        col = self._im2col(X , self.pool_h , self.pool_w)
        col = col.reshape(-1 , self.pool_h * self.pool_w)
        
        arg_max = np.argmax(col , axis = 1)
        A = np.max(col , axis=1)
        A = A.reshape(N , out_h , out_w , C).transpose(0 , 3 , 1 , 2)
        
        self.X = X
        self.arg_max = arg_max
        
        return A
        
    def backward(self , dA):
        dA = dA.transpose(0 , 2 , 3 , 1)

        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dA.size , pool_size))
        dmax[np.arange(self.arg_max.size) , self.arg_max.flatten()] = dA.flatten()
        dmax = dmax.reshape(dA.shape + (pool_size , ))

        dcol = dmax.reshape(dA.shape[0] * dmax.shape[1] * dmax.shape[2] , -1)
        dX = self._col2im(dcol , self.X , self.pool_h , self.pool_w )

        return dX

## 【問題5】平滑化
平滑化するためのFlattenクラスを作成してください。


フォワードのときはチャンネル、高さ、幅の3次元を1次元にreshapeします。その値は記録しておき、バックワードのときに再びreshapeによって形を戻します。


この平滑化のクラスを挟むことで出力前の全結合層に適した配列を作ることができます。

In [8]:
class Flatten:
    
    def forward(self , X):
        self.X = X
        N , C , H , W = self.X.shape
        a = self.X.reshape(N , -1)
        return a 
    
    def backward(self , da):
        x = self.X.shape
        da = da.reshape(x)
        return da

## 【問題6】学習と推定
作成したConv2dを使用してMNISTを学習・推定し、Accuracyを計算してください。


精度は低くともまずは動くことを目指してください。

In [19]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [20]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

In [21]:
from sklearn.model_selection import train_test_split
X_train , X_val , y_train , y_val = train_test_split(
             X_train , y_train_one_hot , test_size = 0.20)

X_train = X_train.reshape(-1 , 1 , 28 , 28)
X_val = X_val.reshape(-1 , 1 , 28 , 28)
X_test = X_test.reshape(-1 , 1 , 28 , 28)

In [22]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練用データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)

    def __len__(self):
        return self._stop

    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        

    def __iter__(self):
        self._counter = 0
        return self

    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [59]:
get_mini_batch = GetMiniBatch(X_train, y_train, batch_size=20)

In [60]:
class FC():
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2 , initializer, optimizer , sigma=0.01):
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.initializer = initializer
        self.sigma = sigma
        
        # 初期化
        self.W = self.initializer.W(self.n_nodes1 , self.n_nodes2)
        self.B = self.initializer.B(self.n_nodes2)
        self.X = None
        self.dW = None
        self.dB = None
        self.hw = 0
        self.hb = 0
        
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """   
        self.X = X
        self.A = np.dot(self.X , self.W) + self.B
        return self.A
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        self.dW = np.dot(self.X.T , dA)
        self.dB = np.sum(dA , axis = 0)
        dZ = np.dot(dA , self.W.T)
        self = self.optimizer.update(self)
        return dZ

In [68]:
def col2im(dcol , X , filter_h , filter_w , stride=1 , pad=0):
    N , C , H , W = X.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = dcol.reshape(N , out_h , out_w , C , filter_h , filter_w).transpose(0,3,4,5,1,2)
    
    img = np.zeros((N , C , H + 2*pad + stride-1 , W + 2*pad + stride-1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[: , : , y:y_max:stride , x:x_max:stride] += col[: , : , y , x , : , :]
            
    return img[: , : , pad:H+pad , pad:W+pad]

In [69]:
class HeInitializer():
    def __init__(self , n_filter , n_channels , filter_h , filter_w):
        self.n_filter = n_filter
        self.n_channels = n_channels
        self.filter_h = filter_h
        self.filter_w  = filter_w
    
    def W(self):
        sigma =  np.sqrt(2.0 / self.n_channels)
        self.W = sigma * np.random.randn(self.n_filter , self.n_channels , self.filter_h , self.filter_w)
        return self.W
    
    def B(self):
        sigma =  np.sqrt(2.0 / self.n_channels)
        self.B = sigma * np.random.randn(self.n_filter)
        return self.B

In [70]:
class SimpleInitializer():

    def __init__(self, sigma):
        self.sigma = sigma
    def W(self, filter_num, filter_size):
        W = self.sigma * np.random.randn(filter_num , filter_size)
        
        return W
    def B(self, filter_size):
        B = self.sigma * np.random.randn(filter_size)
        
        return B

In [71]:
class ReLU():
    def __init__(self):
        self.mask = None
        
    def forward(self , x):
        self.mask = (x <= 0)
        self.A = x.copy()
        self.A[self.mask] = 0
        return self.A
        
    def backward(self , dA):
        dA[self.mask] = 0
        dA = dA
        return dA

In [72]:
class _Conv2d:
    def __init__(self , W , B  , stride=1 , pad=0 , optimizer=AdaGrad(lr=0.01)):
        self.W = W
        self.B = B
        self.stride = stride
        self.pad = pad
        
        self.col = None
        self.col_W = None
        
        self.dW = None
        self.dB = None
        self.hw = 0
        self.hb = 0
        self.optimizer = optimizer

    def _out_h(self , H , FH , pad_h=0 , stride_h=1):
        self.out_h = int(H + 2*pad_h - FH // stride_h+1)
        return self.out_h        

    def _out_w(self , W , FW , pad_w=0 , stride_w=1):
        self.out_w = int(W + 2*pad_w - FW // stride_w+1)
        return self.out_w
    
    def _im2col(self , X , FH , FW):
        N , C , H , W = X.shape
        out_h = self._out_h(H , FH)
        out_w = self._out_w(W , FW)

        img = np.pad(X , [(0,0),(0,0),(self.pad,self.pad),(self.pad,self.pad)] , "constant")
        col = np.zeros((N , C , FH , FW , out_h , out_w))

        for y in range(FH):
            y_max = y + self.stride*out_h
            for x in range(FW):
                x_max = x + self.stride*out_w
                col[: , : , y , x , : , :] = img[: , : , y:y_max:self.stride, x:x_max:self.stride]

        col = col.transpose(0 , 4 , 5 , 1 , 2 , 3).reshape(N*out_h*out_w, -1)
        return col

    def _col2im(self , dcol , X , FH , FW):
        N , C , H , W = self.X.shape
        out_h = self._out_h(H , FH)
        out_w = self._out_w(W , FW)
        col = dcol.reshape(N , out_h , out_W , C , FH , FW).transpose(0,3,4,5,1,2)

        img = np.zeros(N , C , H , 2*self.pad + self.stride-1 , W + 2*self.pad + self.stride-1)
        for y in range(FH):
            y_max = y + self.stride*out_h
            for x in range(FW):
                x_max = x + self.stride*out_w
                img[: , : , y:y_max:self.stride , x:x_max:self.stride] += col[: , : , y , x , : , :]

        return img[: , : , pad:H+pad , pad:W+pad]
    
    def forward(self , X):
        FN , C , FH , FW = self.W.shape
        N , C , H , W = X.shape
        out_h = self._out_h(H , FH)
        out_w = self._out_w(W , FW)
        
        col = self._im2col(X , FH , FW)
        col_W = self.W.reshape(FN , -1).T
        
        A = np.dot(col , col_W) + self.B
        A = A.reshape(N , out_h , out_w , -1).transpose(0 , 3 , 1 , 2)
        self.X = X
        self.col = col
        self.col_W = col_W
        
        return A
    
    def backward(self , dA):
        FN , C , FH , FW = self.W.shape
        dA = dA.transpose(0 , 2 , 3 , 1).reshape(-1 , FN)
        
        self.dB = np.sum(dA , axis = 0)
        self.dW = np.dot(self.col.T , dA)
        self.dW = self.dW.transpose(1 , 0).reshape(FN , C , FH ,FW)
        
        dcol = np.dot(dA , self.col_W.T)
        dX = col2im(dcol , self.X  , FH , FW)
        
        self = self.optimizer.update(self)
        return dX
    
class SGD():
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB
        return layer.W , layer.B
    
class AdaGrad():
    def __init__(self , lr):
        self.lr = lr
        self.hw = None
        self.hb = None
    
    def update(self , layer):
        
        layer.hw += (layer.dW) * (layer.dW)
        layer.W -= self.lr * (layer.dW) / (np.sqrt(layer.hw) + 1e-7)
        layer.hb += (layer.dB) * (layer.dB)
        layer.B -= self.lr * (layer.dB) / (np.sqrt(layer.hb) + 1e-7)
        self.hw = layer.hw
        self.hb = layer.hb
        return layer.W, layer.B  

class SoftmaxWithLoss():
    def __init__(self):
        self.loss = None
        self.y = None
        
        
    def forward(self , x):
        if x.ndim == 2:
            x = x.T
            x = x - np.max(x, axis=0)
            y = np.exp(x) / np.sum(np.exp(x), axis=0)
            return y.T
        x = x - np.max(x)
        return np.exp(x) / np.sum(np.exp(x))
    
    def backward(self , z , y):
        self.dA = z - y
        return self.dA
    
    def Loss(self , z , y):
        loss =  -np.sum(y * np.log(z) + 1e-7) / y.shape[0]
        return loss

In [73]:
class CNN():

    def __init__(self,F_num = 30 ,C =1, FW = 3 , FH = 3 , pool_w = 2, pool_h = 2 ,pad = 0 ,stride = 1 ,
                            epochs = 1,lr = 0.001 ,sigma = 0.01,n_nodes1= 400 , n_nodes2 = 200 , n_output = 10 ,
                             batch_size = 20, verbose = True):
        self.verbose = verbose
        self.sigma = sigma
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.n_output = n_output
        n_features = X_train.shape[1]
        self.n_features = n_features
        self.batch_size = batch_size
        self.lr = lr
        self.loss = []
        self.epochs = epochs
        self.F_num = F_num
        self.C = C
        self.FW = FW
        self.FH = FH
        self.pool_w = pool_w
        self.pool_h = pool_h
        self.pad = pad
        self.stride = stride
        N , C , H , W = X_train.shape
        out_h = int(H + 2*self.pad - self.FH // self.stride+1)
        out_w = int(H + 2*self.pad - self.FH // self.stride+1)
        out_h2 = out_h // self.pool_h
        out_w2 = out_w //  self.pool_w
        self.num = int(self.F_num * C * out_h2 * out_w2)

    def fit(self, X, y, X_val=None, y_val=None):
   
        Xavi = HeInitializer(self.F_num , self.C , self.FW , self.FH)
        self.W = Xavi.W()
        self.B = Xavi.B()
        optimizer = AdaGrad(self.lr)
        self.Con = _Conv2d(self.W,self.B)
        self.pool = MaxPool2D(self.pool_h , self.pool_w)
        self.activation1 = ReLU()
        self.fla   = Flatten()
        self.FC =  FC(self.num  , self.n_output, SimpleInitializer(self.sigma), optimizer)
        self.activation2 = SoftmaxWithLoss()
        
        
        for i in range(self.epochs):
            self.get_mini_batch = GetMiniBatch(X_train, y_train, batch_size=20)
            for self.mini_X_train, self.mini_y_train in get_mini_batch:
                A1 = self.Con.forward(self.mini_X_train)
                A2 = self.pool.forward(A1)
                Z1 = self.activation1.forward(A2)
                F =  self.fla.forward(Z1)
                A3 = self.FC.forward(F)
                Z3 = self.activation2.forward(A3)
                
                dA3 = self.activation2.backward(Z3 , self.mini_y_train) 
                dZ2 = self.FC.backward(dA3)
                dF =  self.fla.backward(dZ2)
                dA2 = self.activation1.backward(dF)
                dA1 = self.pool.backward(dA2)
                dZ1 = self.Con.backward(dA1)
                self.c_loss = self.activation2.Loss(Z3 , self.mini_y_train)
            self.loss = np.append(self.loss , self.c_loss)
        
        if self.verbose:
            print()

    def _predict(self, X):
        A1 = self.Con.forward(X)
        A2 = self.pool.forward(A1)
        Z1 = self.activation1.forward(A2)
        F =  self.fla.forward(Z1)
        A3 = self.FC.forward(F)
        Z3 = self.activation2.forward(A3)
        self.y_pred = np.argmax(Z3 , axis = 1)
        return self.y_pred
    
    def accuracy(self , X , t):
        y_pred = self._predict(X)
        return np.sum(y_pred == t) / float(X.shape[0])

In [ ]:
cnn = CNN(epochs = 5, lr = 0.001)

In [ ]:
cnn.fit(X_train , y_train)

In [78]:
print("Accuracy:{}".format(cnn.accuracy(X_test , y_test)))

Accuracy:0.9711
